# Quantum Teleportation

This notebook demonstrates quantum teleportation. We first use Qiskit's built-in simulator to test our quantum circuit, and then try it out on a real quantum computer.

## The concept

Alice wants to send quantum information to Bob. Specifically, suppose she wants to send the state
$\vert\psi\rangle = \alpha\vert0\rangle + \beta\vert1\rangle$
to Bob. This entails passing on information about $\alpha$ and $\beta$ to Bob.

There exists a theorem in quantum mechanics which states that you cannot simply make an exact copy of an unknown quantum state. This is known as the no-cloning theorem. As a result of this we can see that Alice can't simply generate a copy of $\vert\psi\rangle$ and give the copy to Bob. Copying a state is only possible with a classical computation.

However, by taking advantage of two classical bits and entanglement, Alice can transfer the state $\vert\psi\rangle$ to Bob. We call this teleportation as at the end Bob will have $\vert\psi\rangle$ and Alice won't anymore. Let's see how this works in some detail.

## How does quantum teleportation work?

<img src="images/teleport_theory.png" width="800 px" align="left">

**Step 1**: Alice and Bob create an entangled pair of qubits and each one of them holds on to one of the two qubits in the pair.

**Step 2**: Alice applies a CNOT gate on her entangled qubit, controlled by the qubit she is trying to send Bob. 

**Step 3**: Next, Alice applies a Hadamard gate to the qubit she is trying to send Bob, and applies a measurement to both qubits that she owns.

**Step 4**: Then, it's time for a phone call to Bob. She tells Bob the outcome of her two qubit measurement. Depending on what she says, Bob applies some gates to his entangled qubit, $q_2$. The gates to be applied, based on what Alice says, are as follows :

00 $\rightarrow$ Do nothing

01 $\rightarrow$ Apply $X$ gate

10 $\rightarrow$ Apply $Z$ gate

11 $\rightarrow$ Apply $ZX$ gate

*Note that this transfer of information is classical.*

And voila! At the end of this protocol, Alice's qubit has now teleported to Bob.

## 1. Simulating the teleportation protocol

In [ ]:
# make the imports that are necessary for our work
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute, Aer
from qiskit.tools.visualization import plot_histogram
%matplotlib inline

Do to hardware and measurments constraints we need to implement a slightly modyfied version of the teleportation protocoll to be able to run it on a real QPU. The circuit we will build in this example looks like that

<img src="images/teleport_praxis.png" width="800 px" align="left">

In [ ]:
# Create a Quantum Circuit acting on the quantum and classical register


For this example let us assume that the state Alice likes to teleport to Bob has been generated by applying a Hadamard gate followed by a Z-gate.

In [ ]:
# Alice prepares the state she wants to send to Bob


In [ ]:
#from utils.libquantum import print_sv
#from utils.libqiskit import get_statevector_for_circuit
#print_sv(get_statevector_for_circuit(qc))

In [ ]:
# Next, generate the entangled pair between Alice and Bob 


# Alice prepares her qubits 


# Bob applies final gates according to alice qubits and measures his qubit


# visualize circuit


In [ ]:
# get QASM simulator backend and execute a job with 1024 samples


# get counts and plot histogram


## How will we test this result on a real quantum computer?

If the quantum teleportation circuit works, then at the output of the protocol discussed above will be the same state passed on to Alice. Then, looking at the measurement results that Bob obtains on his qubit we find

In [ ]:
def bobs_qubit_measurements(result):
    return {
            '0': sum([v for k,v in result.items() if k.startswith('0')]),
            '1': sum([v for k,v in result.items() if k.startswith('1')])
            }

In [ ]:
# only have a look at Bob's measaurement results


We see that Bob's qubit is not longer in the state $|0\rangle$ but in an equal superpostion state which Alice prepared. 

Another way to test our algorithm is to undo the applied secret unitary (by applying its conjugate transpose), to yield the $\vert0\rangle$ that we started with. 

We will then do repeated measurements of Bob's qubit to see how many times it gives 0 and how many times it gives 1.

In [ ]:
# remove the last measurement gate from the quantum circuit


In [ ]:
# add reverse secret unitary and measurment


In [ ]:
# visualize the new circuit

In [ ]:
# get QASM simulator backend and execute a job with 1024 samples


# get counts and plot histogram


**Note that the results on the x-axis in the histogram above are ordered as $c_2c_1c_0$. We can see that only results where $c_2 = 0$ appear, indicating that the teleporation protocol has worked.**

In [ ]:
# only hava a look at Bob's measaurement results


## 2. Teleportation on a real quantum computer

In [ ]:
# First, see what devices we are allowed to use by loading our saved accounts
from qiskit import IBMQ

MY_TOKEN = ''
IBMQ.enable_account(token=MY_TOKEN)
provider = IBMQ.get_provider(hub='ibm-q')

In [ ]:
# get the least-busy backend at IBM with 5 qubits
from qiskit.providers.ibmq import least_busy
backend = least_busy(provider.backends(simulator=False, n_qubits=5))

In [ ]:
#execute and monitor job
from qiskit.tools.monitor import job_monitor


In [ ]:
# visualize the results


**As we see here, there are a few results that contain the case when $c_2 = 1$ in a real quantum computer. These arise due to errors in the gates that were applied. Another source of error is the way we're checking for teleportation - we need the series of operators on $q_2$ to be exactly the inverse unitary of those that we applied to $q_0$ at the beginning.**

In contrast, our simulator in the earlier part of the notebook had zero errors in its gates, and allowed error-free teleportation.

In [ ]:
# only hava a look at Bob's measaurement results
